<a href="https://colab.research.google.com/github/agemagician/Prot-Transformers/blob/master/Embedding/Basic/Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h3> Extracting protein sequences' features using ProtBert pretrained-model <h3>

<b>1. Load necessry libraries including huggingface transformers<b>

In [1]:
!pip install -q transformers

     |████████████████████████████████| 675kB 2.8MB/s 
     |████████████████████████████████| 1.1MB 15.7MB/s 
     |████████████████████████████████| 3.8MB 14.3MB/s 
     |████████████████████████████████| 890kB 35.8MB/s 


In [2]:
import torch
from transformers import BertModel, BertTokenizer, pipeline
import re
import numpy as np
import os
import requests
from tqdm.auto import tqdm

<b>2. Set the url location of ProtBert and the vocabulary file<b>

In [3]:
modelUrl = 'https://www.dropbox.com/s/dm3m1o0tsv9terq/pytorch_model.bin?dl=1'
configUrl = 'https://www.dropbox.com/s/d3yw7v4tvi5f4sk/bert_config.json?dl=1'
vocabUrl = 'https://www.dropbox.com/s/jvrleji50ql5m5i/vocab.txt?dl=1'

<b>3. Download ProtBert models and vocabulary files</b>

In [4]:
downloadFolderPath = 'models/ProtBert/'

In [5]:
modelFolderPath = downloadFolderPath

modelFilePath = os.path.join(modelFolderPath, 'pytorch_model.bin')

configFilePath = os.path.join(modelFolderPath, 'config.json')

vocabFilePath = os.path.join(modelFolderPath, 'vocab.txt')

In [6]:
if not os.path.exists(modelFolderPath):
    os.makedirs(modelFolderPath)

In [7]:
def download_file(url, filename):
  response = requests.get(url, stream=True)
  with tqdm.wrapattr(open(filename, "wb"), "write", miniters=1,
                    total=int(response.headers.get('content-length', 0)),
                    desc=filename) as fout:
      for chunk in response.iter_content(chunk_size=4096):
          fout.write(chunk)

In [8]:
if not os.path.exists(modelFilePath):
    download_file(modelUrl, modelFilePath)

if not os.path.exists(configFilePath):
    download_file(configUrl, configFilePath)

if not os.path.exists(vocabFilePath):
    download_file(vocabUrl, vocabFilePath)

<b>4. Load the vocabulary and ProtBert Model<b>

In [9]:
tokenizer = BertTokenizer(vocabFilePath, do_lower_case=False )

In [10]:
model = BertModel.from_pretrained(modelFolderPath)

<b>5. Load the model into the GPU if avilabile<b>

In [11]:
model = pipeline('feature-extraction', model=model, tokenizer=tokenizer,device=0)

<b>6. Create or load sequences and map rarely occured amino acids (U,Z,O,B) to (X)<b>

In [12]:
sequences_Example = ["A E T C Z A O","S K T Z P"]

In [13]:
sequences_Example = [re.sub(r"[UZOB]", "X", sequence) for sequence in sequences_Example]

<b>7. Extracting sequences' features and covert the output to numpy if needed<b>

In [14]:
embedding = model(sequences_Example)

In [15]:
embedding = np.array(embedding)

In [16]:
print(embedding)

[[[ 0.04535879  0.11401558 -0.01174266 ... -0.08750971 -0.11434396
    0.02040843]
  [ 0.09233951  0.13914399 -0.05241546 ... -0.13947977 -0.0428006
    0.07431364]
  [ 0.11505969  0.01998261 -0.08627468 ... -0.00946233 -0.1872724
    0.13169006]
  ...
  [ 0.10792391  0.09771117 -0.05832333 ... -0.12765658 -0.06493297
    0.12894136]
  [ 0.0545795   0.03635893 -0.07821195 ... -0.03016043 -0.06015311
    0.08903892]
  [ 0.05150128  0.0571134  -0.06932486 ... -0.03937564 -0.06630812
    0.09767482]]

 [[ 0.06268476  0.08731148  0.00279815 ... -0.03485441 -0.037174
    0.01748304]
  [ 0.0880546   0.10885102  0.06212992 ...  0.0144938  -0.03327068
   -0.0362049 ]
  [-0.03332657  0.03442192  0.0809955  ...  0.0008738  -0.02733596
    0.05783575]
  ...
  [ 0.00774996  0.03800717 -0.00171148 ... -0.00877388 -0.02430492
    0.07620303]
  [ 0.02335456  0.03041686 -0.00720565 ...  0.00245539 -0.01678618
    0.09118143]
  [ 0.03590559  0.02411232 -0.01498629 ...  0.00813052 -0.00361231
    0.1030

<b>Optional: Remove padding ([PAD]) and special tokens ([CLS],[SEP]) that is added by Bert model<b>

In [17]:
features = [] 

for seq_num in range(len(embedding)):
    seq_len = len(sequences_Example[seq_num].replace(" ", ""))
    start_Idx = 1
    end_Idx = seq_len+1
    seq_emd = embedding[seq_num][start_Idx:end_Idx]
    features.append(seq_emd)

In [18]:
print(features)

[array([[ 0.09233951,  0.13914399, -0.05241546, ..., -0.13947977,
        -0.0428006 ,  0.07431364],
       [ 0.11505969,  0.01998261, -0.08627468, ..., -0.00946233,
        -0.1872724 ,  0.13169006],
       [ 0.04012022,  0.05467906, -0.0424584 , ...,  0.02054804,
        -0.03819017,  0.1011811 ],
       ...,
       [ 0.08853199,  0.0228724 , -0.05121122, ..., -0.0673336 ,
        -0.03013714,  0.04913732],
       [ 0.10792391,  0.09771117, -0.05832333, ..., -0.12765658,
        -0.06493297,  0.12894136],
       [ 0.0545795 ,  0.03635893, -0.07821195, ..., -0.03016043,
        -0.06015311,  0.08903892]]), array([[ 0.0880546 ,  0.10885102,  0.06212992, ...,  0.0144938 ,
        -0.03327068, -0.0362049 ],
       [-0.03332657,  0.03442192,  0.0809955 , ...,  0.0008738 ,
        -0.02733596,  0.05783575],
       [ 0.07896441,  0.10343076, -0.01508527, ...,  0.06711898,
        -0.01733149,  0.12912118],
       [ 0.01805276,  0.07072446, -0.03629201, ..., -0.03547218,
         0.0481908 ,